### store별 tendency 느낌정리
* {2, 3} 얘네는 거의 같음
   > 상관관계가 높은 feature를 살피고 그게 다른 store에도 적용되는지 확인
* {1, 5} 2014년만 다르고 나머지는 거의 같음
   > 1번, 5번 store의 2014년 차이를 조사
* {7} 2013, 2014 매출이 상반기를 제외하고 거의 같음
   > 2012년과도 거의 경향이 같아서 이 때의 feature를 다른 store에도 적용해서 분석해보자
* {4} 2014년 6월달 매출이 드라마틱하게 내려감
   > 이 시점의 날씨를 분석하면 뭔가 나올듯

# 또 느낀 점
1. 월마트가 원하는 건 전체 45개 지점의 날씨와 관련한 예측이다
2. 여기서 문제점, 전체지점을 통합해서 예측한다면 각 지점별 중요한 경향이 무시될 수도 있지 않을까?
3. 제대로 하려면 각 지점에 대한 예측모델을 전부 따로 설정해야 맞지 않을까

1. 스토어마다 비슷한 아이템이 많이 팔린 애들은 같은 지역일 확률이 높다
2. 같은 지역이 아닌데도 스토어마다 비슷한 아이템이 많이 팔리면 그 때 날씨를 추적해서 각 지역간 날씨차이를 살펴본다
3. 이 때 날씨에 따른 item_nbr를 분석한다
3. 12달동안 비슷한 달이 많았던 지역

# 2018 2 14
   * 7번 store를 파다가,
        1. 2013년 7월에 95번 item이 출현
        2. 2013년 3월에 93번 item이 없어짐
        3. 얘네들을 기존의 codesum, 월단위로 item_nbr을 비교하는 함수로만 비교했는데,
       > # codesum만 가지고 뭘 알아내긴 어렵다는 결론에 도달
       > (천둥치고 비내리는 정도의 codesum은 거의 모든 연도,월에 비슷했음)
       
       > # 다른 weather feature를 고려해야 할 것 같다
       
   * 그리고 월단위로 item_nbr를 비교하니 정확히 어느 날짜에 무슨 일이 있었는지 알기가 힘듦
       > 일단위로 item_nbr의 units를 비교해야 함 => __item_nbr_tendency_finely__ 함수작성
       
       > 특정 월의 모든 weather정보를 출력하는 __weather_tendency__ 함수작성   
       
   * ## wetbulb, tavg, BR은 강한 양의 상관관계를 보인다
   
   
   * ## 7번 store 2017년 1월 93번 item, heating과 약한 음의 상관관계를 가지는 듯
   
   * ## 7번 store 2017년 1월 5번, 23번 item의 units이 양의 상관관계를 보인다
       > 5번이 팔리면 23번도 팔림
       
   * jointplot으로 각 feature들 넣었는데 str 에러뜸
       > 알고보니 겉으론 int인데 자료형이 object라 그랬던듯
       
       > astype(int)해주니 잘됨
   
   * 남은 것
       1. weather_tendency 첫 행에 평균값을 가지는 mean row 추가 (vstack)

# 2018 2 15
   * 하나하나 파보는데 45개 store, 열두달에 대해서 언제 다하나 생각이 듦
   * ## 하나하나 하지 말고 아예 모조리 구해버리자
   * # 즉, 한 달을 기준으로 각 item_nbr와 weature feature에 대한 피어슨계수를 구하자
   * # 상관관계를 정량적으로 표현했기 때문에 상관계수가 높은 애들의 특징만 추려보면 된다
   * ## __get_correlation__함수에 구현해놨다
   
   * ## 더 해야할 것
       1. 총 세가지를 구현해야 한다. (item_nbr | weather) (weather | weather) (item | item)
       2. item_nbr | weather는 구현했고 나머지 둘만 더 하면된다
       3. 나머지 둘은 왜하나? wetbulb, tavg, BR 등 날씨끼리, 품목끼리 상관관계가 있는 경우가 있기 때문이다
       
# Missing data 복원 (유클리드 유사도로 거리가 가장 가까운 놈의 값을 갖게 복원?)

# 2018 2 16
   * 상관관계를 pearsonr로 표현할 때 월단위가 아닌 __연단위__로도 구현해보자 = get_correlation_year 함수작성
   * 생각해보니 월구간에 따른 상관관계도 구현하면 좋을듯 __월구간__으로도 구현해보자
   * '0'이 아니라 ' 0'도 문자열로 인식해서 오류가 발생
       > 1. ' ', '-' 없앰
       > 2. 'M', 'T'인 애들은 거르고 계산
   * # 월구간, 연단위로도 pearsonr 계산하도록 get_correlation 함수수정, get_correlation_year 삭제
       > final/07_get_correlation 참조

# 2018 2 17
   * final/07_get_correlation 재업로드, (잘못올렸었음)
   * # 느낀 점
       > codesum이 있다 없다로만 분류해서 정상적인 날이다, 비정상적인 날이다로 판단해도 되는걸까?
       
       > 비가 안오더라도 유난히 습한 날이 있고, 눈이 안내려도 유난히 추운 날이 있을텐뎅
       
       > 그럼에도 불구하고 codesum으로 normal, abnormal을 분류해서
       
       > station_nbr별로 units의 추이를 살펴보는건 대략적인 틀을 잡는데 유의미한 시도
       
       > 단, 각 지역 간 인구수를 정확히 알 수 없기 때문에 좀 더 정확히 하려면 store_nbr별로 시행
       
       > get_correlation과 비교해보자
       
       > # 1. codesum vs salesDifference에서 store_nbr별로 sorting
       
       > # 2. 1번 store에서 50%이상 나는 item_nbr를 get_correlation에서 관찰
       
       > # 3. 샘플 수가 커서 상관계수가 높진 않지만 0에서 많이 먼 값이므로 상관관계가 있음
       
       > (jw/good_correlation.txt 참조)
       
       > # 결론 =  codesum vs salesDifference에서 Difference가 큰 걸 따와서 구체적으로 어떤 feature가 영향을 끼치는지는 get_correlation으로 확인
       
   * codesum만으론 전체적인 틀을 이해하는데 도움이 되지만 뭔가 힘들다.
       > filtering한 weather DataFrame에서 각 feature의 평균을 낸 행을 추가한 후
       
       > 평균에서 멀리 떨어진 날들을 모아서 Difference(%)를 구한다면?
   * get_correlation 수정사항
       1. (store_nbr, year) -> (item_nbr, feature, len(sample)) 로 수정
       2. 거르는 데이터 중에 M, T 말꼬 ''같이 아무것도 없는 데이터도 거름
       
       
   * 여태까지 만든 모든 함수들을 jw_package에 모아놓음
       > from jw_package import *

# 2018 2 18
   * 해야 할 일
       1. 0.01이하인 p-value로 filtering 후 item_nbr별 pearsonr로 sorting
           > https://stackoverflow.com/questions/43960420/sort-python-dictionary-by-absolute-value-of-values
       2. runtime warning 잡기 (전부 missing data인 column때문)

# 2018 3 6
   * 아예 없다 all_missing
    > station에 같이 딸린 나머지 store애들을 전부 넣고 (평균 or best-close)
   * 있긴 있다 partial_missing
    > 그 store의 (평균 or best-close)
    
   * 항상 복원해야 할까?
    > 만약 눈이 아예 안오는 지역은 snowfall을 복원할 필요가 있을까
    
    > 복원하고 안하고의 기준은? 어떤 feature들일까?
---------------------------------------------------------------
---
   * 그 전에 일단 유의한 value가 아니면 전부 np.NaN으로
   
   * NaN으로 바꿀 땐 units가 0인 튜플을 날리면 안됨
    > 가중치를 구할 땐 어차피 가중치도 어차피 0이므로 날리지만
    
    > 가장 가까운 점이나 평균 구할 땐 units와 상관없이 모두 고려되야 함
   
   * ## sunset, sunrise는 연속형 변수가 아니다..
       > 얘넨 빼고 복원
   * # 형변환은 int말고 float

# 2018 3 7
   * partial_missing 끝
   
   * all_missing 해야 되는데 경우가 또 나뉜다
       1. ~~station에 붙은 store가 현재 store말고도 있다~~
            >~~걔네들꺼의 (평균 or 가까운 점)을 쓰면 됨~~
       
       2. station에 붙은 store가 현재 store밖에 없다
            >1, 8, 39, 19, 36, 30, 17은 peer store가 없다
            
            > version A. 다른 모든 station의 점들 중 가장 가까운 점의 다른 feature값을 상속 
            
            > version B. 아니면 그 feature값은 회귀계수 계산에서 뺌
            
            > Version C. 다른 모든 station의 평균 
            # ll_missing은 걍 빼자
-------------
   * 100개의 데이터에 특정 feature가 95개가 missing인데
   intact한 놈들의 평균을 넣었더니 성능이 잘 나오더라
       > 해당 feature가 종속변수에 영향을 별로 안끼쳐서?
       
1. 샘플로 모수를 추정 시 t통계량을 구하는데 n개에 대해 구하면 통계량으로 구한 값이 모수에 살짝 못 미침
2. 그래서 자유도를 고려한 n을 통계량 연산에 쓴다
3. T분포를 쓰는 이유는 모수분산을 모를 때 어쩔수없이 샘플분산을 쓰게 되는데 샘플 분산으로 구한 통계량이 t분포를 따르기 때문이다. 그리고 이 t분포는 샘플 수가 적을때 정규분포를 따르는 통계량보다 추정이 더 정확함
-----------------------------------------------------------------
-----------------------------------------------------------------
### *가장 가까운 점을 상속한다는 의미
1. A day와 B day에 대한 weather 데이터가 있다 치자
2. 각각의 day는 온도와 습도 feature가 있다 치자
3. 예를 들어 우리나라에서 여름이면 온도,습도 둘 다 높을 것이다
4. A day에서 온도,습도가 둘 다 높다 치자
5. B day에선 습도가 missing인데 온도는 높다 치자
6. 온도로 놓고 봤을 때 가장 가까운 점인 A day의 습도로 B day의 습도를 채움

### * 왜 가장 가까운 점을 상속하나
1. 어떤 두 날의 날씨가 비슷하다면 feature도 비슷할 것이다
2. 이를 통해 missing value를 feature가 비슷한 날의 weather로부터 추정할 수 있을 것이다.

### 1. 이 개념엔 오차가 포함되있지 않다..
### 2. 오차를 포함시키려면 어떻게 해야 할까
### 3. 자세힌 모르겠지만 feature를 변수로 가지는 날씨에 대한 pdf는 정규분포를 따를 것 같다
### 4. units ~ feature가 아닌, missing_feature ~ existing feature로도 구해야 하나
### nested regression?

### 5. missing value도 확률론적 방법의 회귀계수를 통해 추정해야 하나?
### 이러면 또 경우가 나뉨. 트레이닝 있어야되는데 store 1번의 depart처럼 아예 전부 missing인 feature들은 다른 store에서 어떤 기준을 세워서 절충하던가 해야함
# 근데 아무경우나 missing value에 대해서 회귀의 회귀를 하는게 아니고 데이터 자체가 확률론적 회귀의 가정을 따라야 한다. 예를 들어 날씨 데이터는 정규분포를 따른다 이런 가정

-----------------------------------------------------------------
-----------------------------------------------------------------
missing_to_nan()
    ~~1. store key weather에 대해서가 아니라 weather에 대해서만~~

가장 가까운 놈으로 filling할 때 기준
    1. station 내의 weather
    2. 가장 비슷한 weather from other stations
    # 만약 각 feature를 Z-score로 비교하려면 해당 feature가 normality test를 통과해야 한다

# 정규분포 안따르더라도 표준점수로 비교가능
~~store key weather를 merge할 때
store에서 units 0인 놈 거르고 merge~~


가장 가까운 놈으로 filling하고 displot찍어보기

-----------------------------------------------------------------
-----------------------------------------------------------------

음 샘플을 기대값으로뺀다음에 편차로나눠주는건 정규화(표준화)이기 때문에

정규분포 모양이 아니라도 상관이없는것같아요

그리고 저렇게 정규화한게 샘플수가 많아질수록 정규분포를 따르구요

-----------------------------------------------------------------
-----------------------------------------------------------------

1. 복원하던 안하던 bimodal distribution이 나온다
2. 봉우리 치솟은 곳이 여름, 겨울의 평균온도같다.
3. 사계절 중 유난히 여름, 겨울이 기니까 그 영향이 반영된 듯
4. 아니면 biased된 가정으로 표본을 뽑아서 그렇게 됬거나

-----------------------------------------------------------------
-----------------------------------------------------------------

전체 station의 데이터를 넣어주니

오른쪽으로 봉우리가 기운 정규분포 모양이 나온다

얘는 뭘까

# 2018 3 12
    1. 데이터들을 yesterday, mean, best close로 복원
    2. 이 데이터들로 실수, 카테고리 변수를 고려하여 선형회귀
    3. 아웃라이어, VIF 10이상인 feature 쳐내고, scale로 성능이 좋아짐
    4. 아웃라이어를 쳐내면 쳐낼수록 성능이 좋아진다 (= 뭐지)
    5. CV(k=4)로 측정해봤을 때 Overfitting이 일어나진 않았음
    (그러므로 정규화를 시켜줄 까닭이 그닥)
    6. 결과를 각 실수 feature에 대해서 종속변수와 plot시켜봄
    7. 뭉뚝한 지렁이 분포를 나타낸다 -> mmung-ttuk 참고
    8. 데이터를 복원할 때 best-close로 했는데 여기서 두가지 생각이 듦
        * best close가 잘못된 접근방식이었다.
        * best close는 원래의 경향대로 복원해서 강화시킬 뿐 경향을 오염시키진 않는다
    9. 그래서 제공받은 데이터 중 intact한 애들만 추려서 봤는데 원래 그렇게 생긴 애들이었다.
    10. 우리가 시도한 best close 복원은 원래의 경향을 강화시킬 뿐이었다
    11. 그리고 실수 weather feature자체가 종속변수와는 아무런 상관이 없다.
    (어떤 feature에 대해 종속변수와 아무런 상관없으면 pvalue고 뭐고 버리자?)
    12. 여기서 질문, 주어진 데이터를 분석한 결과 선형한 관계가 안나온다.
        a. 그러려면 선형한 관계를 나타내는 무언가를 더 추가해야 할까?
        ex) 예를 들면 holiday, weekend, weekday, 공휴일, 특별한 기념일 etc
        b. 주어진 데이터로만 놓고 봤을 때 아무런 관계가 없다고 더 추가해서 찾아내는게 맞는걸까?
        c. 선형한 관계를 유도하여 데이터를 추가하는 시도가 과연 잘하는 짓일까?
    13. 그래서 실수 feaure 전부 날리고 카테고리 feature로만 보니 성능에 아무런 영향이 없었다.
    14. 혹시나 해서 year, month, day를 카테고리 feature로 줘봤는데 오히려 성능이 올랐다.
    15. 원영님이 요일을 나타내는 feature를 추가하셧는데 더 올랐다.
    16. 가만 보면 날씨에는 그닥 영향을 안받지만 물건을 사러 나갈 때 못나갈 정도로 지장있는 날은 영향을 받을 것 같다. 예를 들어 눈이 엄청 많이 오거나 비가 엄청 오거나
    17. 그런 경우는 그 feature를 카테고리화해서 feature로 추가하는건 어떨까?
        ex) 눈이많이왔다/안왔다, 비가많이왔다/안왔다
    

----------

1. Linear regression equation modeling
$$ A*w = b$$
    * A = M X N matrix that row = tuples, column = features
    * w = N vector that weight for each feature
    * b = M vector that label for desired 'units'

# 결론
> 1. ### codesum으로 Differnece(%)를 관찰 후 어떤 feature가 영향을 끼치는 지는 get_correlation으로

> 2. 우리가 해야 할 Linear regression modeling은 아래와 같지 않을까 생각한다
$$ A*w = b$$
    * A = M X N matrix that row = tuples, column = features
    * w = N vector that weight for each feature
    * b = M vector that label for desired 'units'
    
    
    위의 process가 w의 가중치를 구하는데 도움이 되지 않을까 (만약 중간에 논리적인 오류가 없다면)